In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pre_processing as pp
import analysis as anl
import pca

category = 'dvd'

hNeg = True #if true, add negative bigrams for negative reviews
noun = False #if true, add nouns

X, Y, vocabulary = pp.bow(category, hNeg, noun)

print("Vocabulário possui " + str(len(vocabulary)) + " palavras!")

Separa os dados em treinamento e teste:

In [ ]:
# semente usada na randomizacao dos dados.
randomSeed = 10 

# gera os indices aleatorios que irao definir a ordem dos dados
idx_perm = np.random.RandomState(randomSeed).permutation(range(len(Y)))

# ordena os dados de acordo com os indices gerados aleatoriamente
X2, Y2 = X[idx_perm, :], Y[idx_perm]

#X2, Y2 = X[idx_perm, :], Y[idx_perm]

pTrain = 0.8

train_index, test_index = anl.stratified_holdOut(Y, pTrain)

Xtrain, Xval = X2[train_index, :], X2[test_index, :]
Ytrain, Yval = Y2[train_index], Y2[test_index]

Seleciona features com método do chi-quadrado (baseado nos dados de treinamento):

In [ ]:
Xtrain, new_vocabulary, index = pp.chi2(Xtrain, Ytrain, vocabulary)
Xval = Xval[:, index]

In [ ]:
print("Número de features antes do chi-quadrado: " + str(len(vocabulary)))
print("----------------------------------------")
print("Número de features após chi-quadrado: " + str(len(new_vocabulary)))
print(new_vocabulary)

In [ ]:
def calcularProbabilidades(X, Y):
   
    pPos = np.zeros(X.shape[1])
    pNeg = np.zeros(X.shape[1])

    pPos = X[(Y == 1)].sum(axis=0)/sum(Y == 1)
    pNeg = X[(Y == 0)].sum(axis=0)/sum(Y == 0)
    
    probPos = sum(Y==1)/len(Y) 
    probNeg = sum(Y==0)/len(Y)
    
    return probPos, probNeg, pPos, pNeg

In [ ]:
probP, probN, pPos, pNeg = calcularProbabilidades(Xtrain, Ytrain)

In [ ]:
def classificacao(x,probP,probN,pPos,pNeg):

    classe = 0;
    probPos= 0;
    probNeg = 0;

    x = x.toarray()
    
    probPos = probP * (pPos[(x != 0)]).prod() * (1 - pPos[(x == 0)]).prod()
    probNeg = probN * (pNeg[(x != 0)]).prod() * (1 - pNeg[(x == 0)]).prod()
    
    classe = int(probPos >= probNeg)

    return classe

In [ ]:
def predicao(Xval, Yval, probP, probN, pPos, pNeg):
    
    Ypred = []
    for i in range(Xval.shape[0]):
        y = classificacao(Xval[i], probP,probN,pPos,pNeg)
        Ypred.append(y)

    acuracia = np.sum(Ypred==Yval)/len(Yval)
    
    return Ypred, acuracia

In [ ]:
Ypred, acuracia = predicao(Xval, Yval, probP, probN, pPos, pNeg)
print("Acurácia é " + str(acuracia*100))

In [ ]:
classes = np.unique(Y)
auxResults = anl.relatorioDesempenho(Yval, Ypred, classes, imprimeRelatorio=True)

In [ ]:
def curva_aprendizado(X, Y, Xval, Yval):
   
    """
    Funcao usada gerar a curva de aprendizado.
  
    Parametros
    ----------
  
    X : matriz com os dados de treinamento
  
    Y : vetor com as classes dos dados de treinamento
  
    Xval : matriz com os dados de validação
  
    Yval : vetor com as classes dos dados de validação
  
    """

    # inicializa as listas que guardarao a performance no treinamento e na validacao
    perf_train = []
    perf_val = []

    classes = np.unique(Y)
    
    for i in range(10, len(Y)):
        
        probP, probN, pPos, pNeg = calcularProbabilidades(X[:i], Y[:i])
        
        Ypred, acuracia = predicao(X[:i], Y[:i], probP, probN, pPos, pNeg)
        perf_train.append(acuracia)

        Ypred, acuracia = predicao(Xval, Yval, probP, probN, pPos, pNeg)
        perf_val.append(acuracia)


    ##################################################################################
       
    # Define o tamanho da figura 
    plt.figure(figsize=(20,12))

    # Plota os dados
    plt.plot(perf_train, color='blue', linestyle='-', linewidth=1.5, label='Treino') 
    plt.plot(perf_val, color='red', linestyle='-', linewidth=1.5, label='Validação')

    # Define os nomes do eixo x e do eixo y
    plt.xlabel(r'# Qtd. de dados de treinamento',fontsize='x-large') 
    plt.ylabel(r'Acuracia',fontsize='x-large') 

    # Define o título do gráfico
    plt.title(r'Curva de aprendizado', fontsize='x-large')

    # Acrescenta um grid no gráfico
    plt.grid(axis='both')

    # Plota a legenda
    plt.legend()
    
    plt.show()

In [ ]:
#curva_aprendizado(Xtrain, Ytrain, Xval, Yval)